In [1]:
from regain.datasets.hmm import generate_hmm
from regain.hmm.hmm_graphical_lasso import HMM_GraphicalLasso 
import numpy as np
from importlib import reload
import regain.hmm.utils; reload(regain.hmm.utils)
from regain.hmm.utils import results_recap
from regain.hmm.utils import cross_validation#, cross_validation_auto
import pickle as pkl

In [2]:
# list of parameters to generate different kind of datasets

states = [2, 4, 8, 12]
dims = [10, 15, 20, 30, 100]
mode_prec = ['complementary']
mode_mea = ['Normal']


alpha_list = np.logspace(-1, 1, 3)
clusters_list = np.arange(2,15)

In [ ]:
for st in states:
    for dim in dims:
        for mp in mode_prec:
            for mm in mode_mea:
                
                # Generate a dataset
                dataset = generate_hmm(n_samples=1000, n_states=st, n_dim_obs=dim,
                       mode_precisions=mp,mode_mean = mm, transition_type='fixed', sigma=1)
                
                print('Case: N_states:',st,',TS dim:',dim,',Prec mode:',mp,',Mean mode:',mm)
                
                # HMM_GraphicalLasso with a given alpha 
                mdl = HMM_GraphicalLasso(alpha=2, n_clusters=5, verbose=False, mode='scaled',
                                         warm_restart=True, repetitions=3,n_jobs=-1)
                
                #CV
                res = ((st,dim,mp,mm),
                       cross_validation(mdl, dataset['data'], 
                                                       params={'alpha':alpha_list,
                                                       'n_clusters':clusters_list},
                                                         mode='bic', n_repetitions=3)[0],
                      dataset)
                
#                 # HMM_GraphicalLasso with auto generated alpha 
                
#                 mdl = HMM_GraphicalLasso(alpha=2, n_clusters=5, verbose=False, mode='scaled',mode_lam ='auto',
#                                          warm_restart=True, repetitions=3,n_jobs=-1)
                
#                 #CV
#                 res_1 = ((st,dim,mp,mm),cross_validation_auto(mdl, dataset['data'],
#                             params={'n_clusters':clusters_list},mode='stability', n_repetitions=3))
                
#                 with open('one_shot_results_'+st+'states_'+dim+'dimensions_'+mp+'precisiontype_'+mm+'meantype.pkl', wb) as f:
#                     pkl.dump([res, res_1], f)

/home/vero/git_repos/regain/regain/datasets/hmm.py:127: RuntimeWarning: covariance is not positive-semidefinite.
  covariances[state], 1)
  0%|          | 0/13 [00:00<?, ?it/s]

Case: N_states: 2 ,TS dim: 10 ,Prec mode: complementary ,Mean mode: Normal



 46%|████▌     | 6/13 [53:16<1:09:20, 594.38s/it]

In [ ]:
res

In [ ]:
with open("./Results.pkl.pkl", "rb") as f:
    res_fede = pkl.load(f)

In [ ]:
res_alpha_fede = res[1][1]
couples = []
mean_bic = []
std_bic =[]
dispersion_coeff = []
for k in res_alpha_fede.keys():
    couples.append(k)
    mean_bic.append(res_alpha_fede[k]['mean_bic'])
    std_bic.append(res_alpha_fede[k]['std_bic'])
    dispersion_coeff.append(res_alpha_fede[k]['dispersion_coefficient'])
    

In [ ]:
import matplotlib.pyplot as plt 
plt.figure(figsize=(15,5))
plt.plot(mean_bic[:-8])
plt.fill_between(np.arange(len(mean_bic[:-8])), np.array(mean_bic[:-8])-np.array(std_bic[:-8]),
                 np.array(mean_bic[:-8])+np.array(std_bic[:-8]), alpha=0.3)
plt.axvline(np.argmin(mean_bic[:-8]), color='red')
plt.xticks(np.arange(len(mean_bic[:-8])),couples[:-8], rotation='vertical')
plt.show()
plt.figure(figsize=(15,5))
plt.plot(mean_bic)
plt.fill_between(np.arange(len(mean_bic)), np.array(mean_bic)-np.array(std_bic),
                 np.array(mean_bic)+np.array(std_bic), alpha=0.3)
plt.axvline(np.argmin(mean_bic), color='red')
plt.xticks(np.arange(len(mean_bic)),couples, rotation='vertical')
plt.show()


# plt.figure(figsize=(15,5))
# final_res = np.array(mean_bic[:-8]/np.sum(mean_bic[:-8])) + np.array(dispersion_coeff[:-8])
# plt.plot(final_res)
# plt.axvline(np.argmin(final_res), color='red')
# plt.xticks(np.arange(len(mean_bic[:-8])),couples[:-8], rotation='vertical')
# plt.show()

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(dispersion_coeff)
plt.axvline(np.argmin(dispersion_coeff), color='red')
plt.xticks(np.arange(len(mean_bic)),couples, rotation='vertical')
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
final_res = np.array(mean_bic[:-8]) + np.array(dispersion_coeff[:-8])
plt.plot(final_res)
plt.axvline(np.argmin(final_res), color='red')
plt.xticks(np.arange(len(mean_bic[:-8])),couples[:-8], rotation='vertical')
plt.show()

In [ ]:
res_no_alpha_fede = res_[2][1]
couples = []
mean_bic = []
std_bic =[]
dispersion_coeff = []
for k in res_no_alpha_fede.keys():
    couples.append(k)
    mean_bic.append(res_no_alpha_fede[k]['mean_bic'])
    std_bic.append(res_no_alpha_fede[k]['std_bic'])
    dispersion_coeff.append(res_no_alpha_fede[k]['dispersion_coefficient'])

In [ ]:
mean_bic, dispersion_coeff

In [ ]:
import matplotlib.pyplot as plt 
plt.figure(figsize=(15,5))
plt.plot(mean_bic)
plt.fill_between(np.arange(len(mean_bic)), np.array(mean_bic)-np.array(std_bic),
                 np.array(mean_bic)+np.array(std_bic), alpha=0.3)
plt.axvline(np.argmin(mean_bic), color='red')
plt.xticks(np.arange(len(mean_bic)),couples, rotation='vertical')
plt.show()

plt.figure(figsize=(15,10))
plt.plot(dispersion_coeff)
plt.axvline(np.argmin(dispersion_coeff), color='red')
plt.xticks(np.arange(len(mean_bic)),couples, rotation='vertical')
plt.show()

plt.figure(figsize=(15,5))
final_res = np.array(mean_bic/np.sum(mean_bic)) + np.array(dispersion_coeff)
plt.plot(final_res)
plt.axvline(np.argmin(final_res), color='red')
plt.xticks(np.arange(len(mean_bic)),couples, rotation='vertical')
plt.show()

In [ ]:
res_fede

In [ ]:
mdl = HMM_GraphicalLasso(alpha=0.1, n_clusters=4, verbose=False, mode='scaled',
                                         warm_restart=True, repetitions=10,n_jobs=-1)
mdl.fit(dataset['data'])

In [ ]:
from importlib import reload

import regain.hmm.utils; reload(regain.hmm.utils)
from regain.hmm.utils import results_recap
results_recap(dataset['states'], mdl.labels_, dataset['thetas'], mdl.precisions_, 
              dataset['gammas'], mdl.gammas_)